In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
from split_dataset import SplitDataset
from skimage import io
import numpy as np
from scipy.stats import zscore
from bouter.utilities import fast_corrcoef
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpecFromSubplotSpec
from lotr.plotting import add_cbar

In [3]:
path = Path(r'C:\Users\You\Documents\test')

aligned_ds = SplitDataset(path/"rotated")
mask_all = io.imread(path/"mask_rotated.tif", plugin="tifffile")

In [4]:
bin_px = 10

pooled = []
x_pooled = []
y_pooled = []

for plane in range(aligned_ds.shape[1]):
    aligned = aligned_ds[:,plane,:,:]
    mask = mask_all[plane,:,:]
    
    px_mask = np.argwhere(mask>=1)
    px_str = [str(x0)+str(y0) for x0, y0 in px_mask]
    x_loc = np.unique(px_mask[:,0])
    y_loc = np.unique(px_mask[:,1])
    x_bins = np.digitize(x_loc, np.arange(x_loc.min(), x_loc.max()+1, bin_px)) - 1
    y_bins = np.digitize(y_loc, np.arange(y_loc.min(), y_loc.max()+1, bin_px)) - 1
    n_bin_x = x_bins.max() + 1
    n_bin_y = y_bins.max() + 1
    
    traces = np.full((aligned.shape[0], n_bin_x*n_bin_y), np.nan)
    for x in range(n_bin_x):
        for y in range(n_bin_y):
            px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc[y_bins==y]])
            px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
            px_here = px_here[np.isin(px_here_str, px_str)]
            if len(px_here) > 0:
                traces[:, x*n_bin_y+y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
    cor = fast_corrcoef(traces.T)
    mx = np.full((n_bin_x*n_bin_y, n_bin_x*2-1, n_bin_y*2-1), np.nan)
    for i in range(len(cor)):
        if not np.isnan(cor[i,np.arange(len(cor))!=i]).all():
            x0 = i//n_bin_y
            y0 = i%n_bin_y
            for j in range(len(cor)):
                x = j//n_bin_y
                y = j%n_bin_y
                mx[x0*n_bin_y+y0, x-x0+n_bin_x-1, y-y0+n_bin_y-1] = cor[i,j]
    pooled.append(np.nanmean(mx, axis=0))
    
    traces = np.full((aligned.shape[0], n_bin_x), np.nan)
    for x in range(n_bin_x):
        px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            traces[:, x] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
    cor = fast_corrcoef(traces.T)
    mx = np.full((n_bin_x, n_bin_x*2-1), np.nan)
    for i0 in range(len(cor)):
        for i in range(len(cor)):
            mx[i0, i-i0+n_bin_x-1] = cor[i0,i]
    x_pooled.append(np.nanmean(mx, axis=0))

    traces = np.full((aligned.shape[0], n_bin_y), np.nan)
    for y in range(n_bin_y):
        px_here = np.array([(x0, y0) for x0 in x_loc for y0 in y_loc[y_bins==y]])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            traces[:, y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
    cor = fast_corrcoef(traces.T)
    mx = np.full((n_bin_y, n_bin_y*2-1), np.nan)
    for i0 in range(len(cor)):
        for i in range(len(cor)):
            mx[i0, i-i0+n_bin_y-1] = cor[i0,i]
    y_pooled.append(np.nanmean(mx, axis=0))

C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice


In [5]:
x_size = np.array([m.shape[0] for m in pooled]).max()
y_size = np.array([m.shape[1] for m in pooled]).max()

pooled_pad = np.full((len(pooled), x_size, y_size), np.nan)
for i, m in enumerate(pooled):
    x_diff = x_size-m.shape[0]
    y_diff = y_size-m.shape[1]
    pooled_pad[i,:,:] = np.pad(m, ((x_diff//2, x_diff//2), (y_diff//2, y_diff//2)), mode='constant', constant_values=np.nan)
    
x_pooled_pad = np.full((len(pooled), x_size), np.nan)
for i, m in enumerate(x_pooled):
    diff = x_size-m.shape[0]
    x_pooled_pad[i,:] = np.pad(m, diff//2, mode='constant', constant_values=np.nan)
    
y_pooled_pad = np.full((len(pooled), y_size), np.nan)
for i, m in enumerate(y_pooled):
    diff = y_size-m.shape[0]
    y_pooled_pad[i,:] = np.pad(m, diff//2, mode='constant', constant_values=np.nan)

In [6]:
px_size = 0.35
x_ext = (x_size//2)*bin_px*px_size
y_ext = (y_size//2)*bin_px*px_size
mx = np.nanmean(pooled_pad, axis=0)
lim = np.ceil(np.maximum(np.nanmax(mx[mx!=1]), abs(np.nanmin(mx)))*10)/10

fig = plt.figure(figsize=(6, 3))
gs = fig.add_gridspec(2, 2, width_ratios=[11.5,1.5], height_ratios=[3,10], wspace=0, hspace=0)

ax = fig.add_subplot(gs[1,0])
im = ax.imshow(mx.T, vmin=-lim, vmax=lim, extent=(-x_ext, x_ext, -y_ext, y_ext), cmap='RdBu_r', origin="upper")
ax.xaxis.set(ticks_position='bottom')
ax.tick_params(labelsize=6)
ax.set(aspect="equal", xticks=np.linspace(-60, 60, 5), yticks=np.linspace(-30, 30, 3), xlabel="frontal axis [μm]", ylabel="sagittal axis [μm]")
mx_bbox = ax.get_position()
add_cbar(im, ax, inset_loc=(1.1, -0.1, 0.02, 0.15), ticks=(-lim,lim))

ax = fig.add_subplot(gs[0,0])
bbox = ax.get_position()
for i in range(len(x_pooled_pad)):
    ax.plot(x_pooled_pad[i,:], color="gray", alpha=0.2)
ax.plot(np.nanmean(x_pooled_pad, axis=0), c='k')
ax.margins(x=0.01)
ax.set(position=[mx_bbox.x0, mx_bbox.y0+mx_bbox.height, mx_bbox.width, bbox.height])
ax.axis('off')

ax = fig.add_subplot(gs[1,1])
bbox = ax.get_position()
for i in range(len(y_pooled_pad)):
    ax.plot(y_pooled_pad[i,:], range(y_size), color="gray", alpha=0.2)
ax.plot(np.nanmean(y_pooled_pad, axis=0), range(y_size), c='k')
ax.margins(y=0.01)
ax.set(position=[mx_bbox.x0+mx_bbox.width, mx_bbox.y0, bbox.width, mx_bbox.height])
ax.axis('off')

C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.06296659402031904, 1.0506174568581104, -0.2, 20.2)

In [7]:
plt.savefig(path/"Fig5d.pdf")